##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regularización de grafos para clasificación de sentimientos con grafos sintetizados

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
  <td>     <a href="https://tfhub.dev/"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo de TF Hub</a>
</td>
</table>

## Descripción general

En este bloc de notas se clasifican reseñas de películas como *positiva* o *negativa* a partir del texto de la reseña. Este es un ejemplo de clasificación *binaria*, un tipo de problema de aprendizaje automático importante y ampliamente aplicable.

Demostraremos el uso de la regularización de grafos mediante la creación de un grafo a partir de una entrada dada. A continuación, compartimos la receta general para crear un modelo de grafo regularizado con el marco de trabajo del aprendizaje estructurado neuronal (NSL) cuando la entrada no contenga un grafo explícito:

1. Crear incorporaciones para cada muestra de texto de la entrada. Se puede hacer con modelos previamente entrenados como [word2vec](https://arxiv.org/pdf/1310.4546.pdf), [Swivel](https://arxiv.org/abs/1602.02215), [BERT](https://arxiv.org/abs/1810.04805), etc.
2. Crear un grafo basado en estas incorporaciones usando una métrica de similitud, como las de distancia 'L2', distancia 'coseno', etc. Los nodos del grafo corresponden a las muestras del grafo, y las aristas, a la similitud entre pares de muestras.
3. Generar datos de entrenamiento a partir del grafo anterior sintetizado y de las características de la muestra. Los datos de entrenamiento que resulten de esta acción contendrán las características vecinas, además de las características del nodo original.
4. Crear con una API secuencial, funcional o de subclase de Keras una red neuronal como modelo de base.
5. Encapsular el modelo de base con la clase del encapsulador GraphRegularization, provista por el marco de trabajo NSL, para crear un nuevo modelo Keras de grafo. El modelo nuevo incluirá una pérdida por regularización de grafo, como el término de regularización en su objetivo de entrenamiento.
6. Entrenar y evaluar el modelo Keras de grafo.

**Nota**: Previmos que a los lectores les tomará alrededor de una hora leer este tutorial completo.

## Requisitos

1. Instalar el paquete de aprendizaje estructurado neuronal.
2. Instalar tensorflow-hub.

In [ ]:
!pip install --quiet neural-structured-learning
!pip install --quiet tensorflow-hub

## Dependencias e importaciones

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import neural_structured_learning as nsl

import tensorflow as tf
import tensorflow_hub as hub

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## Conjunto de datos de IMDB

El [conjunto de datos de IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) contiene el texto de 50 000 reseñas de películas de [Internet Movie Database](https://www.imdb.com/). Se divide en 25 000 reseñas para entrenamiento y 25 000 para prueba. Los conjuntos de entrenamiento y prueba están *equilibrados*, lo que significa que contienen la misma cantidad de reseñas positivas y negativas.

En este tutorial usaremos una versión previamente procesada del conjunto de datos IMDB.

### Descargar el conjunto de datos de IMDB preprocesado

El conjunto de datos de IMDB viene empaquetado con TensorFlow. Ya ha sido previamente procesado, de modo tal que las reseñas (secuencias de palabras) se han convertido en secuencias de enteros, donde cada entero representa a una palabra específica del diccionario.

Con el siguiente código se descarga el conjunto de datos de IMDB (o, si ya se ha descargado, se usa una copia en caché):

In [ ]:
imdb = tf.keras.datasets.imdb
(pp_train_data, pp_train_labels), (pp_test_data, pp_test_labels) = (
    imdb.load_data(num_words=10000))

El argumento `num_words=10000` contiene las 10 000 palabras más frecuentes en los datos de entrenamiento. Las palabras menos frecuentes se descartan para que el vocabulario mantenga un tamaño que pueda seguir siendo gestionable.

### Exploración de los datos

Tomémonos un momento para comprender el formato de los datos. El conjunto de datos viene previamente procesado: cada ejemplo es un arreglo de enteros que representan las palabras de la reseña de una película. La etiqueta es un valor de número entero que puede ser 0 o 1, donde 0 corresponde a una reseña negativa y 1 corresponde a una positiva.

In [ ]:
print('Training entries: {}, labels: {}'.format(
    len(pp_train_data), len(pp_train_labels)))
training_samples_count = len(pp_train_data)

El texto de las reseñas se ha convertido en enteros, donde cada entero representa a una palabra específica del diccionario. A continuación, compartimos cómo se ve la primera reseña:

In [ ]:
print(pp_train_data[0])

Las reseñas de películas pueden tener longitudes diferentes. El siguiente código muestra la cantidad de palabras que contienen la primera revisión y la segunda. Como las entradas de una red neuronal deben tener la misma longitud, más adelante deberemos resolver este aspecto.

In [ ]:
len(pp_train_data[0]), len(pp_train_data[1])

### Conversión de enteros de vuelta en palabras

Puede resultar útil saber cómo volver a convertir los enteros en el texto correspondiente. Lo que haremos será crear una función ayudante para consultar un objeto diccionario que contenga el entero para el mapeo de la <em>string</em>:

In [ ]:
def build_reverse_word_index():
  # A dictionary mapping words to an integer index
  word_index = imdb.get_word_index()

  # The first indices are reserved
  word_index = {k: (v + 3) for k, v in word_index.items()}
  word_index['<PAD>'] = 0
  word_index['<START>'] = 1
  word_index['<UNK>'] = 2  # unknown
  word_index['<UNUSED>'] = 3
  return dict((value, key) for (key, value) in word_index.items())

reverse_word_index = build_reverse_word_index()

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

Ahora, podemos usar la función `decode_review` para mostrar el texto para la primera reseña:

In [ ]:
decode_review(pp_train_data[0])

## Construcción de grafos

La construcción de grafos consiste en crear incorporaciones (<em>embeddings</em>) para muestras de texto y después, usar una función de similitud para comparar dichas incorporaciones.

Antes de continuar, crearemos un directorio para compartir los artefactos creados en este tutorial.

In [ ]:
!mkdir -p /tmp/imdb

### Creación de incorporaciones de muestra

Usaremos las incorporaciones Swivel previamente entrenadas para crear otras incorporaciones con el formato `tf.train.Example` para cada muestra de la entrada. Almacenaremos las incorporaciones resultantes con el formato `TFRecord` junto con una característica adicional que representa al ID de cada muestra. Es importante y nos permitirá hacer coincidir, más adelante, a las incorporaciones de la muestra con los nodos correspondientes del grafo.

In [ ]:
pretrained_embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'

hub_layer = hub.KerasLayer(
    pretrained_embedding, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
def _int64_feature(value):
  """Returns int64 tf.train.Feature."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value.tolist()))


def _bytes_feature(value):
  """Returns bytes tf.train.Feature."""
  return tf.train.Feature(
      bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))


def _float_feature(value):
  """Returns float tf.train.Feature."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value.tolist()))


def create_embedding_example(word_vector, record_id):
  """Create tf.Example containing the sample's embedding and its ID."""

  text = decode_review(word_vector)

  # Shape = [batch_size,].
  sentence_embedding = hub_layer(tf.reshape(text, shape=[-1,]))

  # Flatten the sentence embedding back to 1-D.
  sentence_embedding = tf.reshape(sentence_embedding, shape=[-1])

  features = {
      'id': _bytes_feature(str(record_id)),
      'embedding': _float_feature(sentence_embedding.numpy())
  }
  return tf.train.Example(features=tf.train.Features(feature=features))


def create_embeddings(word_vectors, output_path, starting_record_id):
  record_id = int(starting_record_id)
  with tf.io.TFRecordWriter(output_path) as writer:
    for word_vector in word_vectors:
      example = create_embedding_example(word_vector, record_id)
      record_id = record_id + 1
      writer.write(example.SerializeToString())
  return record_id


# Persist TF.Example features containing embeddings for training data in
# TFRecord format.
create_embeddings(pp_train_data, '/tmp/imdb/embeddings.tfr', 0)

### Creación de un grafo

Ahora que tenemos las incorporaciones de la muestra, las usaremos para crear un grafo de similitud. Es decir, los nodos del grafo corresponderán a las muestras, y las aristas, a la similitud entre pares de nodos.

El aprendizaje estructurado neuronal proporciona una biblioteca de creación de grafos, que permite crear un grafo en base a las incorporaciones de las muestras. Utiliza la [**similitud coseno**](https://en.wikipedia.org/wiki/Cosine_similarity) como medida de similitud para comparar las incorporaciones y crear aristas entre ellas. También sirve para descartar aristas distintas del grafo final. En este ejemplo, al utilizar 0.99 como umbral de similitud y 12345 como semilla aleatoria, terminamos con un grafo que tiene 429 415 aristas bidireccionales. Aquí usamos el soporte del "constructor" de grafos para [locality-sensitive hashing](https://en.wikipedia.org/wiki/Locality-sensitive_hashing) (LSH), a fin de acelerar la creación del grafo. Para más detalles sobre cómo utilizar el soporte LSH del constructor de grafos, consulte la documentación de la API [`build_graph_from_config`](https://www.tensorflow.org/neural_structured_learning/api_docs/python/nsl/tools/build_graph_from_config).

In [ ]:
graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=0.99, lsh_splits=32, lsh_rounds=15, random_seed=12345)
nsl.tools.build_graph_from_config(['/tmp/imdb/embeddings.tfr'],
                                  '/tmp/imdb/graph_99.tsv',
                                  graph_builder_config)

Cada arista bidireccional está representada, en el archivo TSV de salida, por dos aristas dirigidas; de modo tal, que contiene un total de 429 415 × 2 = 858 830 líneas:

In [ ]:
!wc -l /tmp/imdb/graph_99.tsv

**Nota:** La calidad del grafo y, por extensión, la calidad de la incorporación son muy importantes para la regularización del grafo. Si bien hemos usado incorporaciones Swivel en este bloc de notas, si usamos incorporaciones BERT, por ejemplo, probablemente capturaremos con mucha mayor precisión la semántica de las reseñas. Recomendamos a los usuarios utilizar las incorporaciones que prefieran y las que sean adecuadas a sus necesidades.

## Características de muestra

Creamos características de muestra para nuestro problema con el formato `tf.train.Example` y persistimos con ellas en el formato `TFRecord`. Cada muestra incluirá las siguientes tres características:

1. Un **id** (ID): el ID del nodo de la muestra.
2. **words** (Palabras): una lista int64 que contiene los ID de las palabras.
3. Una **label** (etiqueta): un patrón singleton int64 con el que se identifica la clase de destino de la reseña.

In [ ]:
def create_example(word_vector, label, record_id):
  """Create tf.Example containing the sample's word vector, label, and ID."""
  features = {
      'id': _bytes_feature(str(record_id)),
      'words': _int64_feature(np.asarray(word_vector)),
      'label': _int64_feature(np.asarray([label])),
  }
  return tf.train.Example(features=tf.train.Features(feature=features))

def create_records(word_vectors, labels, record_path, starting_record_id):
  record_id = int(starting_record_id)
  with tf.io.TFRecordWriter(record_path) as writer:
    for word_vector, label in zip(word_vectors, labels):
      example = create_example(word_vector, label, record_id)
      record_id = record_id + 1
      writer.write(example.SerializeToString())
  return record_id

# Persist TF.Example features (word vectors and labels) for training and test
# data in TFRecord format.
next_record_id = create_records(pp_train_data, pp_train_labels,
                                '/tmp/imdb/train_data.tfr', 0)
create_records(pp_test_data, pp_test_labels, '/tmp/imdb/test_data.tfr',
               next_record_id)

## Datos de entrenamiento aumentado con grafos vecinos

Como tenemos las características de las muestras y el grafo sintetizado, podemos generar datos de entrenamiento aumentado para aprendizaje estructurado neuronal. El marco NSL ofrece una biblioteca que permite combinar las características del grafo y de la muestra para producir los datos finales de entrenamiento necesarios para la regularización del grafo. Los datos resultantes del entrenamiento incluirán las características de la muestra original y las de sus correspondientes vecinos.

Para este tutorial, consideramos aristas sin dirección y usamos un máximo de 3 vecinos por muestra, para datos de entrenamiento aumentado con vecinos de grafos.

In [ ]:
nsl.tools.pack_nbrs(
    '/tmp/imdb/train_data.tfr',
    '',
    '/tmp/imdb/graph_99.tsv',
    '/tmp/imdb/nsl_train_data.tfr',
    add_undirected_edges=True,
    max_nbrs=3)

## Modelo base

Ahora, está todo listo para crear un modelo base sin regularización de grafos. Para construir este modelo, podemos usar las incorporaciones que se utilizaron para la creación del grafo o, aprender incorporaciones nuevas junto con la tarea de clasificación. Para cumplir con lo propuesto para este bloc de notas, haremos lo último.

### Variables globales

In [ ]:
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'

### Hiperparámetros

Utilizaremos una instancia de `HParams` para incluir varios hiperparámetros y contantes que se usan para el entrenamiento y la evaluación. A continuación, describimos brevemente cada una de ellas:

- **num_classes**: hay 2 clases: una *positiva* y otra *negativa*.

- **max_seq_length**: es la cantidad máxima de palabras consideradas de cada reseña de película, en este ejemplo.

- **vocab_size**: es el tamaño del vocabulario considerado para este ejemplo.

- **distance_type**: es la métrica de distancia utilizada para regularizar la muestra con sus vecinas.

- **graph_regularization_multiplier**: controla el peso relativo del término de regularización del grafo en la función de pérdida general.

- **num_neighbors**: la cantidad de vecinos utilizados para la regularización del grafo. Este valor tiene que ser menor o igual que el argumento `max_nbrs` utilizado arriba cuando invocamos `nsl.tools.pack_nbrs`.

- **num_fc_units**: la cantidad de unidades de una capa totalmente conectada de la red neuronal.

- **train_epochs**: la cantidad de épocas de entrenamiento.

- **batch_size**: el tamaño del lote para entrenamiento y evaluación.

- **eval_steps**: la cantidad de lotes a procesar antes de que la evaluación se complete. Si se determina como `None`, se evalúan todas las instancias del conjunto de prueba.

In [ ]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### dataset parameters
    self.num_classes = 2
    self.max_seq_length = 256
    self.vocab_size = 10000
    ### neural graph learning parameters
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 2
    ### model architecture
    self.num_embedding_dims = 16
    self.num_lstm_dims = 64
    self.num_fc_units = 64
    ### training parameters
    self.train_epochs = 10
    self.batch_size = 128
    ### eval parameters
    self.eval_steps = None  # All instances in the test set are evaluated.

HPARAMS = HParams()

### Preparación de los datos

Las reseñas (arreglos de los enteros) deben convertirse a tensores antes de ingresar a la red neuronal. Esta conversión se puede hacer de dos maneras:

- Convirtiendo los arreglos en vectores de los `0` y los `1`, indicando la ocurrencia de palabras, en forma similar a una codificación en un solo paso <em>one-hot</em>. Por ejemplo, la secuencia `[3, 5]` se convertiría en un vector `10000`-dimensional que tiene todos ceros, excepto por los índices `3` y `5`, que son unos. Después, usemos esto como la primera capa de nuestra red, una capa `Dense` que puede trabajar con datos de vectores de punto flotante. Esta opción es intensiva con respecto a la memoria, a pesar de que requiere una matriz con el tamaño `num_words * num_reviews`.

- Como alternativa, podemos rellenar (<em>pad</em>) los arreglos para que todos tengan la misma longitud y después, crear un tensor entero de forma `max_length * num_reviews`. Podemos usar una capa de incorporación capaz de operar con esta forma como primera capa de nuestra red.

En este tutorial, usaremos la segunda opción.

Dado que las reseñas de películas deben tener la misma longitud, usaremos la función `pad_sequence` definida a continuación para estandarizar las longitudes.

In [ ]:
def make_dataset(file_path, training=False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """

  def pad_sequence(sequence, max_seq_length):
    """Pads the input sequence (a `tf.SparseTensor`) to `max_seq_length`."""
    pad_size = tf.maximum([0], max_seq_length - tf.shape(sequence)[0])
    padded = tf.concat(
        [sequence.values,
         tf.fill((pad_size), tf.cast(0, sequence.dtype))],
        axis=0)
    # The input sequence may be larger than max_seq_length. Truncate down if
    # necessary.
    return tf.slice(padded, [0], [max_seq_length])

  def parse_example(example_proto):
    """Extracts relevant fields from the `example_proto`.

    Args:
      example_proto: An instance of `tf.train.Example`.

    Returns:
      A pair whose first value is a dictionary containing relevant features
      and whose second value contains the ground truth labels.
    """
    # The 'words' feature is a variable length word ID vector.
    feature_spec = {
        'words': tf.io.VarLenFeature(tf.int64),
        'label': tf.io.FixedLenFeature((), tf.int64, default_value=-1),
    }
    # We also extract corresponding neighbor features in a similar manner to
    # the features above during training.
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i,
                                         NBR_WEIGHT_SUFFIX)
        feature_spec[nbr_feature_key] = tf.io.VarLenFeature(tf.int64)

        # We assign a default value of 0.0 for the neighbor weight so that
        # graph regularization is done on samples based on their exact number
        # of neighbors. In other words, non-existent neighbors are discounted.
        feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
            [1], tf.float32, default_value=tf.constant([0.0]))

    features = tf.io.parse_single_example(example_proto, feature_spec)

    # Since the 'words' feature is a variable length word vector, we pad it to a
    # constant maximum length based on HPARAMS.max_seq_length
    features['words'] = pad_sequence(features['words'], HPARAMS.max_seq_length)
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        features[nbr_feature_key] = pad_sequence(features[nbr_feature_key],
                                                 HPARAMS.max_seq_length)

    labels = features.pop('label')
    return features, labels

  dataset = tf.data.TFRecordDataset([file_path])
  if training:
    dataset = dataset.shuffle(10000)
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  return dataset


train_dataset = make_dataset('/tmp/imdb/nsl_train_data.tfr', True)
test_dataset = make_dataset('/tmp/imdb/test_data.tfr')

### Creación del modelo

Una red neuronal se crea apilando capas, lo que implica tomar dos decisiones principales en términos de arquitectura:

- ¿Cuántas capas se usarán en el modelo?
- ¿Cuántas *unidades ocultas* se usarán para cada capa?

En este ejemplo, los datos de entrada son un arreglo de índices de palabras. Las etiquetas que se deben predecir son 0 o 1.

Para este tutorial utilizaremos una LSTM bidireccional como modelo base.

In [ ]:
# This function exists as an alternative to the bi-LSTM model used in this
# notebook.
def make_feed_forward_model():
  """Builds a simple 2 layer feed forward neural network."""
  inputs = tf.keras.Input(
      shape=(HPARAMS.max_seq_length,), dtype='int64', name='words')
  embedding_layer = tf.keras.layers.Embedding(HPARAMS.vocab_size, 16)(inputs)
  pooling_layer = tf.keras.layers.GlobalAveragePooling1D()(embedding_layer)
  dense_layer = tf.keras.layers.Dense(16, activation='relu')(pooling_layer)
  outputs = tf.keras.layers.Dense(1)(dense_layer)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


def make_bilstm_model():
  """Builds a bi-directional LSTM model."""
  inputs = tf.keras.Input(
      shape=(HPARAMS.max_seq_length,), dtype='int64', name='words')
  embedding_layer = tf.keras.layers.Embedding(HPARAMS.vocab_size,
                                              HPARAMS.num_embedding_dims)(
                                                  inputs)
  lstm_layer = tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(HPARAMS.num_lstm_dims))(
          embedding_layer)
  dense_layer = tf.keras.layers.Dense(
      HPARAMS.num_fc_units, activation='relu')(
          lstm_layer)
  outputs = tf.keras.layers.Dense(1)(dense_layer)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


# Feel free to use an architecture of your choice.
model = make_bilstm_model()
model.summary()

Las capas se apilan secuencialmente con efectividad para generar el clasificador:

1. La primera es una capa de `Input` que toma el vocabulario codificado con enteros.
2. La primera capa es una capa `Embedding`. Esta capa toma las reseñas cifradas con números enteros y busca el vector de incorporación para cada índice de palabra. Estos vectores se aprenden a medida que se entrena el modelo. Los vectores agregan una dimensión al arreglo de salida. Las dimensiones resultantes son las siguientes: `(batch, sequence, embedding)`.
3. Después, una capa LSTM bidireccional devuelve un vector de salida con longitud fija para cada ejemplo.
4. Este vector de salida de longitud fija se canaliza a través de una capa (`Dense`) completamente conectada con 64 unidades ocultas.
5. La última capa está conectada densamente con un nodo de salida único. Cuando utilizamos la función de activación `sigmoid`, este valor es un flotante de entre 0 y 1 que representa una probabilidad o un nivel de confianza.

### Unidades ocultas

El modelo anterior tiene dos capas intermedias u "ocultas" entre la entrada y la salida, excluyendo la capa `Embedding`. La cantidad de salidas (unidades, nodos o neuronas) es la dimensión del espacio representativo para la capa. En otras palabras, la cantidad de libertad que se le permite a la capa cuando aprende una representación interna.

Si un modelo tiene más unidades ocultas (un espacio de representación con más dimensiones) o más capas, la red puede aprender representaciones más complejas. Sin embargo, la red se vuelve computacionalmente más cara y puede derivar en patrones indeseados de aprendizaje; patrones que mejoren el desempeño de los datos de entrenamiento pero no de los datos de prueba. A esto se lo denomina "sobreajuste" (*overfitting*).

### Función de pérdida y optimizador

Un modelo necesita una función de pérdida y un optimizador para el entrenamiento. Dado que este es un problema de clasificación binaria y el modelo genera una probabilidad (una capa de una sola unidad con una activación sigmoide), usaremos la función de pérdida `binary_crossentropy`.

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Crear un conjunto de validación

Durante el entrenamiento, nos conviene comprobar la exactitud del modelo con datos que no haya visto antes. Creemos un *conjunto de validación*. Para crearlo separemos una fracción de los datos de entrenamiento originales. (¿Por qué no usamos el conjunto de prueba ahora? Nuestro objetivo es desarrollar y ajustar el modelo usando solamente los datos de entrenamiento y después, usar los datos de prueba una sola vez para evaluar la exactitud).

En este tutorial, tomamos aproximadamente el 10 % de las muestras de entrenamiento iniciales (10% de 25000) como datos etiquetados para el entrenamiento y, el resto como datos de validación. Como la separación inicial entre entrenamiento o prueba se hizo 50/50 (25000 muestras cada opción), ahora, una separación efectiva entre entrenamiento, prueba y validación tiene la siguiente proporción: 5/45/50.

Tenga en cuenta que el 'train_dataset' ya ha sido agrupado en lotes y aleatorizado. 

In [ ]:
validation_fraction = 0.9
validation_size = int(validation_fraction *
                      int(training_samples_count / HPARAMS.batch_size))
print(validation_size)
validation_dataset = train_dataset.take(validation_size)
train_dataset = train_dataset.skip(validation_size)

### Entrenamiento del modelo

Entrenemos el modelo en minilotes. Mientras se entrena, supervisemos la pérdida y la exactitud del modelo en el conjunto de validación:

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=HPARAMS.train_epochs,
    verbose=1)

### Evaluación del modelo

Ahora, veamos el desempeño del modelo. Nos devolverá dos valores; la pérdida (un número que representa nuestro error, los valores bajos son mejores) y la exactitud.

In [ ]:
results = model.evaluate(test_dataset, steps=HPARAMS.eval_steps)
print(results)

### Creación de un grafo de exactitud y pérdida a lo largo del tiempo

`model.fit()` devuelve un objeto `History` que contiene un diccionario con todo lo que pasó durante el entrenamiento:

In [ ]:
history_dict = history.history
history_dict.keys()

Hay cuatro entradas: una por cada métrica que se monitoreó durante el entrenamiento y la validación. Podemos usarlas para trazar la pérdida y validación del entrenamiento, para compararlas. Podemos hacer lo mismo con la exactitud de entrenamiento y validación:

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "-r^" is for solid red line with triangle markers.
plt.plot(epochs, loss, '-r^', label='Training loss')
# "-b0" is for solid blue line with circle markers.
plt.plot(epochs, val_loss, '-bo', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.plot(epochs, val_acc, '-bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

Como puede ver, la pérdida del entrenamiento *se reduce* época tras época y la exactitud del entrenamiento *aumenta* a medida que pasan las épocas. Esto es lo que suele pasar cuando se usa una optimización con descenso de gradiente, debe reducir al mínimo la cantidad deseada en cada iteración.

## Regularización de grafos

No estamos listos para probar la regularización de grafos con el modelo de base que acabamos de crear. Usaremos la clase del encapsulador `GraphRegularization` proporcionada por el marco de aprendizaje estructurado neuronal para encapsular el modelo (bi-LSTM) de base, a fin de incluir la regularización del grafo. El resto de los pasos para entrenamiento y evaluación del modelo del grafo regularizado son similares a los del modelo de base.

### Creación de un modelo con grafo regularizado

Para evaluar el beneficio incremental de la regularización de grafos, crearemos una nueva instancia del modelo de base. El motivo es que el `model` ya ha sido entrenado para unas pocas iteraciones y reusarlo para crear un modelo de grafo regularizado no ofrecería un elemento de comparación verdadero para el `model`.

In [ ]:
# Build a new base LSTM model.
base_reg_model = make_bilstm_model()

In [ ]:
# Wrap the base model with graph regularization.
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=HPARAMS.num_neighbors,
    multiplier=HPARAMS.graph_regularization_multiplier,
    distance_type=HPARAMS.distance_type,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Entrenamiento del modelo

In [ ]:
graph_reg_history = graph_reg_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=HPARAMS.train_epochs,
    verbose=1)

### Evaluación del modelo

In [ ]:
graph_reg_results = graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)
print(graph_reg_results)

### Creación de un grafo de exactitud y pérdida a lo largo del tiempo

In [ ]:
graph_reg_history_dict = graph_reg_history.history
graph_reg_history_dict.keys()

En el diccionario, hay cinco entradas en total: la pérdida del entrenamiento, la exactitud del entrenamiento, la pérdida del grafo de entrenamiento, la pérdida de validación y la exactitud de validación. Para compararlas, podemos trazarlas todas juntas. Tenga en cuenta que la pérdida del grafo solamente se computa durante el entrenamiento.

In [ ]:
acc = graph_reg_history_dict['accuracy']
val_acc = graph_reg_history_dict['val_accuracy']
loss = graph_reg_history_dict['loss']
graph_loss = graph_reg_history_dict['scaled_graph_loss']
val_loss = graph_reg_history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.clf()   # clear figure

# "-r^" is for solid red line with triangle markers.
plt.plot(epochs, loss, '-r^', label='Training loss')
# "-gD" is for solid green line with diamond markers.
plt.plot(epochs, graph_loss, '-gD', label='Training graph loss')
# "-b0" is for solid blue line with circle markers.
plt.plot(epochs, val_loss, '-bo', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.plot(epochs, val_acc, '-bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

## El poder del aprendizaje semisupervisado

El aprendizaje semisupervisado y, más específicamente, la regularización del grafo en el contexto de este tutorial puede ser realmente potente cuando la cantidad de datos de entrenamiento es poca. La falta de datos de entrenamiento se compensa aprovechando la similitud que haya entre las muestras de entrenamiento; algo que no es posible con el entrenamiento supervisado tradicional.

Definimos a la ***relación de supervisión*** como la relación de las muestras de entrenamiento con respecto a la cantidad total de muestras (incluidas las muestras de entrenamiento, validación y prueba). En este bloc de notas hemos aplicado una relación de supervisión de 0.05 (es decir, del 5% de los datos etiquetados) para entrenar ambos modelos, el de base y el de grafo regularizado. En la celda a continuación, ilustramos el impacto que tiene la relación de supervisión en la exactitud del modelo.

In [ ]:
# Accuracy values for both the Bi-LSTM model and the feed forward NN model have
# been precomputed for the following supervision ratios.

supervision_ratios = [0.3, 0.15, 0.05, 0.03, 0.02, 0.01, 0.005]

model_tags = ['Bi-LSTM model', 'Feed Forward NN model']
base_model_accs = [[84, 84, 83, 80, 65, 52, 50], [87, 86, 76, 74, 67, 52, 51]]
graph_reg_model_accs = [[84, 84, 83, 83, 65, 63, 50],
                        [87, 86, 80, 75, 67, 52, 50]]

plt.clf()  # clear figure

fig, axes = plt.subplots(1, 2)
fig.set_size_inches((12, 5))

for ax, model_tag, base_model_acc, graph_reg_model_acc in zip(
    axes, model_tags, base_model_accs, graph_reg_model_accs):

  # "-r^" is for solid red line with triangle markers.
  ax.plot(base_model_acc, '-r^', label='Base model')
  # "-gD" is for solid green line with diamond markers.
  ax.plot(graph_reg_model_acc, '-gD', label='Graph-regularized model')
  ax.set_title(model_tag)
  ax.set_xlabel('Supervision ratio')
  ax.set_ylabel('Accuracy(%)')
  ax.set_ylim((25, 100))
  ax.set_xticks(range(len(supervision_ratios)))
  ax.set_xticklabels(supervision_ratios)
  ax.legend(loc='best')

plt.show()

Se puede observar que a medida que la relación de supervisión disminuye, la exactitud del modelo también baja. Esto les sucede tanto al modelo de base como al de grafo regularizado, independientemente de la arquitectura que se aplique para cada modelo. Sin embargo, es de destacar que el modelo de grafo regularizado se desempeña mejor que el base en ambas arquitecturas. En particular, en el caso del modelo Bi-LSTM cuando la relación de supervisión es de 0.01, la exactitud del modelo de grafo regularizado es **~20 %** superior a la del modelo de base. Esto se debe, principalmente, al aprendizaje semisupervisado del modelo de grafo regularizado; donde, además de las muestras de entrenamiento, se usa la similitud estructural entre las mismas.

## Conclusión

Hemos demostrado el uso de la regularización de grafos con el marco de aprendizaje estructurado neuronal (NSL), incluso cuando la entrada no contiene un grafo explícito. Para el caso, consideramos la tarea de clasificación de sentimientos de reseñas sobre películas de IMDB para la que sintetizamos un grafo de similitud basado en incorporaciones de reseñas. A los usuarios, les recomendamos que practiquen con más experiencias, variando los hiperparámetros, la cantidad de supervisión y las diferentes arquitecturas de los modelos.